In [1]:
import os, gc, cv2, torch, h5py, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from transformers import AutoImageProcessor, ZoeDepthForDepthEstimation

# ========== 设备 ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========== 模型 ==========
model_name = "Intel/zoedepth-nyu-kitti"
processor = AutoImageProcessor.from_pretrained(model_name)
model = ZoeDepthForDepthEstimation.from_pretrained(model_name).to(device)
model.eval()

# ========== 数据路径 ==========
mat_path = '/kaggle/input/nyuv2-dataset/nyu_depth_v2_labeled.mat'
idx_list_path = '/kaggle/input/nyu2-test/nyu2_test.txt'
save_root = '/kaggle/working'
os.makedirs(save_root, exist_ok=True)

# 可视化输出
vis_dir = os.path.join(save_root, 'visualizations_zoe_nyu')
os.makedirs(vis_dir, exist_ok=True)

# 预测输出（新增）
pred_root = os.path.join(save_root, 'predictions_zoe_nyu')
pred_metric_dir = os.path.join(pred_root, 'metric')
pred_aligned_dir = os.path.join(pred_root, 'aligned')
os.makedirs(pred_metric_dir, exist_ok=True)
os.makedirs(pred_aligned_dir, exist_ok=True)

# ========== 读 NYUv2 ==========
with h5py.File(mat_path, 'r') as f:
    images = f['images'][:]
    depths = f['depths'][:]

images = np.transpose(images, (3, 2, 1, 0))
depths = np.transpose(depths, (2, 1, 0))
idx_list = np.loadtxt(idx_list_path).astype(int) - 1  # 转为 0-based

# ========== 工具函数 ==========
def save_vis(rgb, gt, pred_metric, pred_aligned, mask, save_path):
    plt.figure(figsize=(20, 5))

    valid_base = mask & (gt > 0) & np.isfinite(gt)
    valid_m = valid_base & np.isfinite(pred_metric)
    valid_a = valid_base & np.isfinite(pred_aligned)

    depths = []
    if np.any(valid_m): depths += [gt[valid_m], pred_metric[valid_m]]
    if np.any(valid_a): depths += [gt[valid_a], pred_aligned[valid_a]]
    if len(depths) == 0:
        vmin, vmax = 0, 1
    else:
        depths = np.concatenate(depths)
        vmin, vmax = np.percentile(depths, 2), np.percentile(depths, 98)

    plt.subplot(1, 4, 1)
    plt.imshow(rgb); plt.title("RGB"); plt.axis("off")

    plt.subplot(1, 4, 2)
    plt.imshow(gt, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("GT Depth"); plt.colorbar(fraction=0.046, pad=0.04); plt.axis("off")

    plt.subplot(1, 4, 3)
    plt.imshow(pred_metric, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("Predicted (Metric)"); plt.colorbar(fraction=0.046, pad=0.04); plt.axis("off")

    plt.subplot(1, 4, 4)
    plt.imshow(pred_aligned, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("Predicted (Aligned)"); plt.colorbar(fraction=0.046, pad=0.04); plt.axis("off")

    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()

def compute_abs_rel(pred, gt, mask):
    valid = mask & (gt > 0)
    pred, gt = pred[valid], gt[valid]
    return np.mean(np.abs(pred - gt) / gt) if len(gt) > 0 else np.nan

def compute_delta1(pred, gt, mask):
    valid = mask & (gt > 1e-6) & (pred > 1e-6)
    pred, gt = pred[valid], gt[valid]
    thresh = np.maximum(pred / gt, gt / pred)
    return np.mean(thresh < 1.25) if len(thresh) > 0 else np.nan

def scale_match(pred, gt, mask):
    pred, gt = pred[mask], gt[mask]
    denom = np.sum(pred ** 2)
    return (np.sum(gt * pred) / denom) if denom > 1e-8 else 1.0

# ========== 主评测 ==========
results = []

for idx in tqdm(idx_list, desc="Evaluating"):
    try:
        # 取出数据
        rgb = images[:, :, :, idx].astype(np.uint8)
        gt_depth = depths[:, :, idx].astype(np.float32)
        mask = (gt_depth > 0) & np.isfinite(gt_depth)

        # 前处理
        input_image = Image.fromarray(rgb)
        inputs = processor(images=input_image, return_tensors="pt").to(device)

        # 前向
        with torch.no_grad():
            outputs = model(**inputs)

        # 后处理到原图尺寸
        H, W = rgb.shape[0], rgb.shape[1]
        processed = processor.post_process_depth_estimation(outputs, source_sizes=[(H, W)])
        pred_depth = processed[0]["predicted_depth"].squeeze().detach().cpu().numpy().astype(np.float32)

        # 尺寸对齐
        if pred_depth.shape != gt_depth.shape:
            pred_depth = cv2.resize(pred_depth, (gt_depth.shape[1], gt_depth.shape[0]), interpolation=cv2.INTER_LINEAR)

        # ===== Metric（不对齐） =====
        abs_rel_metric = compute_abs_rel(pred_depth, gt_depth, mask)
        delta1_metric = compute_delta1(pred_depth, gt_depth, mask)

        # ===== Relative（逐图对齐） =====
        scale = scale_match(pred_depth, gt_depth, mask)
        pred_aligned = (pred_depth * scale).astype(np.float32)
        abs_rel_rel = compute_abs_rel(pred_aligned, gt_depth, mask)
        delta1_rel = compute_delta1(pred_aligned, gt_depth, mask)

        results.append([int(idx),
                        float(abs_rel_metric), float(delta1_metric),
                        float(abs_rel_rel), float(delta1_rel)])

        print(f"[{idx}] Metric: AbsRel={abs_rel_metric:.4f}, δ1={delta1_metric*100:.2f}% | "
              f"Relative: AbsRel={abs_rel_rel:.4f}, δ1={delta1_rel*100:.2f}%")

        # ===== 保存预测（新增） =====
        # 原始预测（Metric）
        np.save(os.path.join(pred_metric_dir, f"{int(idx):04d}.npy"), pred_depth)
        # 对齐后的预测（Relative）
        np.save(os.path.join(pred_aligned_dir, f"{int(idx):04d}.npy"), pred_aligned)

        # 可视化（保存 Relative 对齐版本）
        save_path = os.path.join(vis_dir, f"{int(idx):04d}_zoe_nyu.png")
        save_vis(rgb, gt_depth, pred_depth, pred_aligned, mask, save_path)

        del rgb, gt_depth, pred_depth, pred_aligned, inputs, outputs, processed
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

    except Exception as e:
        print(f"跳过 {idx}，错误: {e}")
        continue

# ========== 保存结果 ==========
df = pd.DataFrame(results, columns=["idx", "AbsRel_metric", "Delta1_metric",
                                    "AbsRel_relative", "Delta1_relative"])
mean_row = ["mean"] + df.iloc[:, 1:].mean().tolist()
df.loc[len(df)] = mean_row

csv_path = os.path.join(save_root, "zoe_nyuv2_results.csv")
df.to_csv(csv_path, index=False)
print(f"评估结果已保存：{csv_path}")

# ========== 打包可视化与预测（新增） ==========
zip_vis_path = shutil.make_archive(os.path.join(save_root, "zoe_nyuv2_visualizations"),'zip', root_dir=vis_dir)
print(f"可视化图像已打包: {zip_vis_path}")

zip_metric_path = shutil.make_archive(os.path.join(save_root, "zoe_nyuv2_pred_metric"),'zip', root_dir=pred_metric_dir)
print(f"原始预测(未对齐)已打包: {zip_metric_path}")

zip_aligned_path = shutil.make_archive(os.path.join(save_root, "zoe_nyuv2_pred_aligned"),'zip', root_dir=pred_aligned_dir)
print(f"对齐预测(Aligned)已打包: {zip_aligned_path}")


2025-08-12 22:28:34.003468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755037714.244783      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755037714.314294      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

...

[1444] Metric: AbsRel=0.1846, δ1=65.40% | Relative: AbsRel=0.1996, δ1=64.19%


Evaluating: 100%|█████████▉| 665/666 [39:08<00:05,  5.47s/it]

[1447] Metric: AbsRel=0.1300, δ1=86.65% | Relative: AbsRel=0.1537, δ1=83.60%


Evaluating: 100%|██████████| 666/666 [39:13<00:00,  3.53s/it]


评估结果已保存：/kaggle/working/zoe_nyuv2_results.csv
可视化图像已打包: /kaggle/working/zoe_nyuv2_visualizations.zip
原始预测(未对齐)已打包: /kaggle/working/zoe_nyuv2_pred_metric.zip
对齐预测(Aligned)已打包: /kaggle/working/zoe_nyuv2_pred_aligned.zip
